<a href="https://colab.research.google.com/github/gatienc/multimodal_product_data_classification/blob/main/notebooks/gatien_fusion_model_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!pip install unidecode

#Idea
idea of this fusion model: our 2 model gives a vector of 27 element as output,
we want to freeze both pretrained model, and just combine the output to classify better than on by one

il faudrait qu'il y ai une couche qui fasse : $a*out_{image}+b*out_{text}$
avec a et b des params entraînables

In [79]:
BATCH_SIZE=275#10.4/15GO
num_classes=27

train_percentage=0.80
test_percentage=0.10
valid_percentage=0.10


# Imports


In [80]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision


from tqdm.auto import tqdm # for cool loading bar


# import for NLP
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical

from unidecode import unidecode
from nltk.corpus import stopwords



from sklearn.metrics import f1_score

import zipfile
import os
import copy

import matplotlib.pyplot as plt


In [82]:
# mount the drive where your dataset is availabledevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

if not os.path.exists('datasets'):
  os.makedirs('datasets')
  with zipfile.ZipFile(filepath+'images.zip', 'r') as zip_ref:
      zip_ref.extractall('datasets')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Preprocessing

In [89]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")


In [90]:
# Cleaning and Preprocessing Text


CLEANR = re.compile('<.*?>') # delete html tag

def clean_html(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
def clean_special(text):
    text = re.sub(r'[^a-z% ]', '', text)
    return(text)

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    text=re.sub('\d', '%', text)#replace all digits by %
    text = unidecode(text)
    text=clean_html(text)
    text=clean_special(text)
    #clean stopwords
    text_list=text.split()
    return([word for word in text_list if word not in final_stopwords_list])



final_stopwords_list = stopwords.words('english') + stopwords.words('french')+stopwords.words('german')# Fr,En,De Stopwords


# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)
X_train['description'] = X_train['description'].fillna('').apply(clean_text)


In [91]:
pd.set_option('display.max_colwidth', 150)
X_train.designation.head(50)


0                                                                 [olivia, personalisiertes, notizbuch, %%%, seiten, punktraster, ca, din, a%, rosendesign]
1     [journal, arts, ndeg, %%%, %%%%%%%%, lart, marche, salon, dart, asiatique, paris, jacques, barrere, francois, perrier, reforme, ventes, encheres, ...
2                                                                    [grand, stylet, ergonomique, bleu, gamepad, nintendo, wii, u, speedlink, pilot, style]
3                                                                                           [peluche, donald, europe, disneyland, %%%%, marionnette, doigt]
4                                                                                                                                          [guerre, tuques]
5                                                                                 [afrique, contemporaine, ndeg, %%%, hiver, %%%%, dossier, japon, afrique]
6                                                               

In [65]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
sequences = tokenizer.texts_to_sequences(X_train['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)

# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)# nearly 70 000 of vocab size, it seems too much

# Split data into training and validation set (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(data, y_train, test_size=0.2,shuffle=False)
y_train=y_train["prdtypecode"].tolist()
y_val=y_val["prdtypecode"].tolist()

# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)

66304



# Data loading

In [66]:
class ImageTextDataLoader():
    """Title, Description and Image dataset."""

    def __init__(self, dataframe, image_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            image_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir,self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return [self.df,sample]


    def image_search(self,id):
        """
        Return the path of the image from the productID

        still not optimized o(n**2) (did'nt found better way)
        """

        def wich_patterns(patterns_list,string):
            """
            returns the the pattern (from patterns) that match string if exists
            """
            for index,pattern in enumerate(patterns_list):
                if pattern in string:
                    return([True, index])
            else: return([False,""])

        if type(id)==int:
            id=[id]

        directory=self.image_dir+"/images"
        #get the list of string patterns to look for in the file of the folder
        patterns=np.ones(range(len(id)))
        for i in range(len(id)):
            patterns.append("_"+str(id)+".")

        arr_str = [''] * len(id)

        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                print(file_path)
                with open(file_path, 'r') as f:
                    in_pattern,pattern_index= wich_patterns(patterns,os.path.abspath(file_path))
                    if in_pattern:
                        arr_str[pattern_index]=os.path.abspath(file_path)
        if "" not in arr_str:
            return(arr_str)
        else :
            raise ValueError("the array string is not fully filled, maybe an image description is lacking in the file, or problem in the dataloader")



In [67]:
transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(256),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
       ])

In [68]:
dataset=ImageTextDataLoader("/content/drive/MyDrive/datasets/multimodal_product_classification/X_train.csv","/content/drive/MyDrive/datasets/images/image_train",transform)

In [69]:
train_size = int(train_percentage * len(dataset))
valid_size = int(test_percentage * len(dataset))
test_size = len(dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)


dataloaders={
    'train':train_dataloader,
    'val':valid_dataloader,
    'test':test_dataloader
    }

model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

# Replace the last fully-connected layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes) # num_classes is the number of classes in your dataset


model.load_state_dict(torch.load("/content/drive/MyDrive/Lessons/Models/multimodal_classification/Resnet16-best10.ckpt",map_location=device))


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 79.6MB/s]


<All keys matched successfully>

In [70]:
data_batch = next(iter(train_dataloader))

AttributeError: 'ImageTextDataLoader' object has no attribute 'landmarks_frame'

In [ ]:
data_batch